In [ ]:
# Sync BOMs from Odoo V14 to V18 by deleting and recreating them

import os
import xmlrpc.client
from dotenv import load_dotenv

# Load environment variables
load_dotenv(override=True)

# Get environment variables for V14
JUSTFRAMEIT_ODOO_V14_URL = os.getenv('JUSTFRAMEIT_ODOO_V14_URL')
JUSTFRAMEIT_ODOO_V14_DB = os.getenv('JUSTFRAMEIT_ODOO_V14_DB')
JUSTFRAMEIT_ODOO_V14_USERNAME = os.getenv('JUSTFRAMEIT_ODOO_V14_USERNAME')
JUSTFRAMEIT_ODOO_V14_API_KEY = os.getenv('JUSTFRAMEIT_ODOO_V14_API_KEY')

# Get environment variables for V18
JUSTFRAMEIT_ODOO_URL = os.getenv('JUSTFRAMEIT_ODOO_URL_TEST')
JUSTFRAMEIT_ODOO_DB = os.getenv('JUSTFRAMEIT_ODOO_DB_TEST')
JUSTFRAMEIT_ODOO_USERNAME = os.getenv('JUSTFRAMEIT_ODOO_USERNAME')
JUSTFRAMEIT_ODOO_API_KEY = os.getenv('JUSTFRAMEIT_ODOO_API_KEY')

print("=" * 80)
print("Syncing BOMs from Odoo V14 to V18")
print("=" * 80)

try:
    # Authenticate to V14
    print("\n[V14] Authenticating...")
    common_v14 = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_V14_URL}/xmlrpc/2/common", allow_none=True)
    uid_v14 = common_v14.authenticate(JUSTFRAMEIT_ODOO_V14_DB, JUSTFRAMEIT_ODOO_V14_USERNAME, JUSTFRAMEIT_ODOO_V14_API_KEY, {})
    print(f"✅ V14 Authenticated! UID: {uid_v14}")
    
    # Authenticate to V18
    print("\n[V18] Authenticating...")
    common_v18 = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/common", allow_none=True)
    uid_v18 = common_v18.authenticate(JUSTFRAMEIT_ODOO_DB, JUSTFRAMEIT_ODOO_USERNAME, JUSTFRAMEIT_ODOO_API_KEY, {})
    print(f"✅ V18 Authenticated! UID: {uid_v18}\n")
    
    # Create models proxies
    models_v14 = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_V14_URL}/xmlrpc/2/object", allow_none=True)
    models_v18 = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/object", allow_none=True)
    
    # Define specific category IDs to check
    target_category_ids = [4, 8, 10, 57, 82]
    target_category_ids_v18 = [129, 130, 131, 133, 135] #for testing
    #target_category_ids = [10] #for testing
    
    print(f"Target categories: {target_category_ids}\n")
    
    # Get category details from V14 (single call)
    categories = models_v14.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid_v14, JUSTFRAMEIT_ODOO_V14_API_KEY,
        'product.category', 'read',
        [target_category_ids],
        {'fields': ['display_name']})
    
    print("Categories to sync:")
    for cat in categories:
        print(f"  - Category ID {cat['id']}: {cat['display_name']}")
    print()
    
    # Get all product templates in these categories from V14 (single call)
    print("[V14] Fetching product templates in these categories...")
    product_template_ids = models_v14.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid_v14, JUSTFRAMEIT_ODOO_V14_API_KEY,
        'product.template', 'search',
        [[['categ_id', 'in', target_category_ids]]])
    
    print(f"Found {len(product_template_ids)} product templates\n")
    
    # Search for BOMs with is_custom_made = False in V14 (single call)
    print("[V14] Searching for BOMs with is_custom_made = False...")
    print("=" * 80)
    
    bom_ids_v14 = models_v14.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid_v14, JUSTFRAMEIT_ODOO_V14_API_KEY,
        'mrp.bom', 'search',
        [[['product_tmpl_id', 'in', product_template_ids], ['is_custom_made', '=', False]]])
    
    print(f"\n[V14] Found {len(bom_ids_v14)} BOMs with is_custom_made = False\n")
    
    if not bom_ids_v14:
        print("No BOMs found with is_custom_made = False in V14")
    else:
        # Get ALL BOM details from V14 in a single call
        boms_v14 = models_v14.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid_v14, JUSTFRAMEIT_ODOO_V14_API_KEY,
            'mrp.bom', 'read',
            [bom_ids_v14],
            {'fields': ['id', 'code', 'display_name', 'product_tmpl_id', 'product_id', 'is_custom_made', 'bom_line_ids', 'product_qty']})
        
        # Get ALL BOM lines from V14 in a single call
        all_bom_line_ids_v14 = []
        for bom in boms_v14:
            if bom['bom_line_ids']:
                all_bom_line_ids_v14.extend(bom['bom_line_ids'])
        
        bom_lines_v14_dict = {}
        if all_bom_line_ids_v14:
            all_bom_lines_v14 = models_v14.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid_v14, JUSTFRAMEIT_ODOO_V14_API_KEY,
                'mrp.bom.line', 'read',
                [all_bom_line_ids_v14],
                {'fields': ['id', 'product_id', 'product_qty', 'bom_id']})
            
            # Get all product IDs from BOM lines to fetch product codes
            product_ids_v14 = [line['product_id'][0] for line in all_bom_lines_v14 if line['product_id']]
            
            # Fetch product codes for all products in V14
            products_v14 = models_v14.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid_v14, JUSTFRAMEIT_ODOO_V14_API_KEY,
                'product.product', 'read',
                [product_ids_v14],
                {'fields': ['id', 'product_code', 'display_name']})
            
            # Create a mapping of product_id to product data
            product_data_map_v14 = {p['id']: {'code': p['product_code'], 'name': p['display_name']} for p in products_v14}
            
            # Organize by bom_id for easy lookup
            for line in all_bom_lines_v14:
                bom_id = line['bom_id'][0] if line['bom_id'] else None
                if bom_id not in bom_lines_v14_dict:
                    bom_lines_v14_dict[bom_id] = []
                # Add product_code to the line data
                product_id = line['product_id'][0] if line['product_id'] else None
                product_data = product_data_map_v14.get(product_id, {'code': 'N/A', 'name': 'N/A'})
                line['product_code'] = product_data['code']
                line['product_name'] = product_data['name']
                bom_lines_v14_dict[bom_id].append(line)
        
        # Step 1: Delete existing BOMs in V18 for these categories
        print("\n" + "=" * 80)
        print("[V18] STEP 1: Deleting existing BOMs in target categories...")
        print("=" * 80)
        
        # Get product templates in V18 for these categories
        product_template_ids_v18 = models_v18.execute_kw(JUSTFRAMEIT_ODOO_DB, uid_v18, JUSTFRAMEIT_ODOO_API_KEY,
            'product.template', 'search',
            [[['categ_id', 'in', target_category_ids_v18]]])
        
        # Find BOMs for these product templates in V18
        bom_ids_v18_to_delete = models_v18.execute_kw(JUSTFRAMEIT_ODOO_DB, uid_v18, JUSTFRAMEIT_ODOO_API_KEY,
            'mrp.bom', 'search',
            [[['product_tmpl_id', 'in', product_template_ids_v18]]])
        
        print(f"Found {len(bom_ids_v18_to_delete)} BOMs to delete in V18")
        
        deleted_count = 0
        failed_to_delete = []
        
        for bom_id in bom_ids_v18_to_delete:
            try:
                models_v18.execute_kw(JUSTFRAMEIT_ODOO_DB, uid_v18, JUSTFRAMEIT_ODOO_API_KEY,
                    'mrp.bom', 'unlink',
                    [[bom_id]])
                deleted_count += 1
                print(f"  ✅ Deleted BOM ID: {bom_id}")
            except Exception as e:
                failed_to_delete.append({'id': bom_id, 'error': str(e)})
                print(f"  ❌ Failed to delete BOM ID {bom_id}: {e}")
        
        print(f"\nDeleted {deleted_count} BOMs successfully")
        if failed_to_delete:
            print(f"Failed to delete {len(failed_to_delete)} BOMs")
        
        # Step 2: Create product code mapping in V18
        print("\n" + "=" * 80)
        print("[V18] STEP 2: Building product code mapping...")
        print("=" * 80)
        
        # Get all products in V18 with their codes
        all_products_v18 = models_v18.execute_kw(JUSTFRAMEIT_ODOO_DB, uid_v18, JUSTFRAMEIT_ODOO_API_KEY,
            'product.product', 'search_read',
            [[]],
            {'fields': ['id', 'x_studio_product_code', 'display_name', 'product_tmpl_id']})
        
        # Create mapping: product_code -> product_id in V18
        product_code_to_id_v18 = {}
        for p in all_products_v18:
            if p['x_studio_product_code']:
                product_code_to_id_v18[p['x_studio_product_code']] = {
                    'id': p['id'],
                    'name': p['display_name'],
                    'tmpl_id': p['product_tmpl_id'][0] if p['product_tmpl_id'] else None
                }
        
        print(f"Mapped {len(product_code_to_id_v18)} products by code in V18")
        
        # Step 3: Recreate BOMs in V18
        print("\n" + "=" * 80)
        print("[V18] STEP 3: Recreating BOMs from V14...")
        print("=" * 80)
        
        created_count = 0
        failed_to_create = []
        created_products = []
        
        for bom_v14 in boms_v14:
            print(f"\n{'=' * 80}")
            print(f"Processing BOM ID (V14): {bom_v14['id']}")
            print(f"  Reference: {bom_v14['code']}")
            print(f"  Display Name: {bom_v14['display_name']}")
            
            # Get product template ID in V18
            product_tmpl_id_v14 = bom_v14['product_tmpl_id'][0] if bom_v14['product_tmpl_id'] else None
            
            if not product_tmpl_id_v14:
                print(f"  ❌ No product template in V14, skipping")
                failed_to_create.append({'bom_id': bom_v14['id'], 'error': 'No product template'})
                continue
            
            # Get product template details from V14
            product_tmpl_v14 = models_v14.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid_v14, JUSTFRAMEIT_ODOO_V14_API_KEY,
                'product.template', 'read',
                [[product_tmpl_id_v14]],
                {'fields': ['default_code', 'name']})
            
            if not product_tmpl_v14:
                print(f"  ❌ Could not read product template from V14, skipping")
                failed_to_create.append({'bom_id': bom_v14['id'], 'error': 'Could not read product template'})
                continue
            
            product_code_v14 = product_tmpl_v14[0]['default_code']
            
            # Find corresponding product template in V18 by code
            product_tmpl_v18_ids = models_v18.execute_kw(JUSTFRAMEIT_ODOO_DB, uid_v18, JUSTFRAMEIT_ODOO_API_KEY,
                'product.template', 'search',
                [[['default_code', '=', product_code_v14]]])
            
            if not product_tmpl_v18_ids:
                print(f"  ❌ Product template not found in V18 (code: {product_code_v14}), skipping")
                failed_to_create.append({'bom_id': bom_v14['id'], 'error': f'Product template not found in V18 (code: {product_code_v14})'})
                continue
            
            product_tmpl_id_v18 = product_tmpl_v18_ids[0]
            
            # Handle product variant if specified
            product_id_v18 = False
            if bom_v14['product_id']:
                product_id_v14 = bom_v14['product_id'][0]
                # Get product variant details from V14
                product_v14 = models_v14.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid_v14, JUSTFRAMEIT_ODOO_V14_API_KEY,
                    'product.product', 'read',
                    [[product_id_v14]],
                    {'fields': ['product_code']})
                
                if product_v14 and product_v14[0]['product_code']:
                    variant_code = product_v14[0]['product_code']
                    # Find variant in V18
                    product_v18_ids = models_v18.execute_kw(JUSTFRAMEIT_ODOO_DB, uid_v18, JUSTFRAMEIT_ODOO_API_KEY,
                        'product.product', 'search',
                        [[['x_studio_product_code', '=', variant_code]]])
                    
                    if product_v18_ids:
                        product_id_v18 = product_v18_ids[0]
            
            # Prepare BOM lines
            bom_lines_v14 = bom_lines_v14_dict.get(bom_v14['id'], [])
            bom_line_vals = []
            
            print(f"  Processing {len(bom_lines_v14)} BOM lines...")
            
            for line in bom_lines_v14:
                component_code = line['product_code']
                component_name = line['product_name']
                component_qty = line['product_qty']
                
                # Find component in V18
                if component_code in product_code_to_id_v18:
                    component_id_v18 = product_code_to_id_v18[component_code]['id']
                    print(f"    ✅ Found component: {component_name} (Code: {component_code})")
                else:
                    # Component doesn't exist, create it
                    print(f"    ⚠️  Component not found in V18: {component_name} (Code: {component_code})")
                    print(f"       Creating component...")
                    
                    try:
                        # Get full product details from V14
                        product_id_v14 = line['product_id'][0] if line['product_id'] else None
                        product_details_v14 = models_v14.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid_v14, JUSTFRAMEIT_ODOO_V14_API_KEY,
                            'product.product', 'read',
                            [[product_id_v14]],
                            {'fields': ['name', 'product_code', 'categ_id', 'type', 'uom_id', 'uom_po_id']})
                        
                        if product_details_v14:
                            product_detail = product_details_v14[0]
                            
                            # Create product in V18
                            new_product_vals = {
                                'name': product_detail['name'],
                                'x_studio_product_code': component_code,
                                'type': product_detail.get('type', 'product'),
                            }
                            
                            # Add category if exists in V18
                            if product_detail.get('categ_id'):
                                categ_id_v14 = product_detail['categ_id'][0]
                                # Try to find same category in V18
                                categ_v18_ids = models_v18.execute_kw(JUSTFRAMEIT_ODOO_DB, uid_v18, JUSTFRAMEIT_ODOO_API_KEY,
                                    'product.category', 'search',
                                    [[['id', '=', categ_id_v14]]])
                                if categ_v18_ids:
                                    new_product_vals['categ_id'] = categ_v18_ids[0]
                            
                            component_id_v18 = models_v18.execute_kw(JUSTFRAMEIT_ODOO_DB, uid_v18, JUSTFRAMEIT_ODOO_API_KEY,
                                'product.product', 'create',
                                [new_product_vals])
                            
                            # Update mapping
                            product_code_to_id_v18[component_code] = {
                                'id': component_id_v18,
                                'name': product_detail['name'],
                                'tmpl_id': None
                            }
                            
                            created_products.append({
                                'id': component_id_v18,
                                'code': component_code,
                                'name': product_detail['name']
                            })
                            
                            print(f"       ✅ Created component ID: {component_id_v18}")
                        else:
                            print(f"       ❌ Could not read product details from V14, skipping component")
                            continue
                    except Exception as e:
                        print(f"       ❌ Failed to create component: {e}")
                        continue
                
                # Add BOM line
                bom_line_vals.append((0, 0, {
                    'product_id': component_id_v18,
                    'product_qty': component_qty,
                }))
            
            # Create BOM in V18
            try:
                # Use the exact same reference as V14
                bom_reference = bom_v14['code'] if bom_v14['code'] else ''
                
                bom_vals = {
                    'product_tmpl_id': product_tmpl_id_v18,
                    'code': bom_reference,
                    'product_qty': bom_v14.get('product_qty', 1.0),
                    'bom_line_ids': bom_line_vals,
                }
                
                if product_id_v18:
                    bom_vals['product_id'] = product_id_v18
                
                new_bom_id = models_v18.execute_kw(JUSTFRAMEIT_ODOO_DB, uid_v18, JUSTFRAMEIT_ODOO_API_KEY,
                    'mrp.bom', 'create',
                    [bom_vals])
                
                # Post V14 BOM ID in the chatter
                chatter_message = f"Synced from V14 BOM ID: {bom_v14['id']}"
                models_v18.execute_kw(JUSTFRAMEIT_ODOO_DB, uid_v18, JUSTFRAMEIT_ODOO_API_KEY,
                    'mrp.bom', 'message_post',
                    [new_bom_id],
                    {
                        'body': chatter_message,
                        'message_type': 'comment',
                        'subtype_xmlid': 'mail.mt_note'
                    })
                
                created_count += 1
                print(f"  ✅ Created BOM in V18 with ID: {new_bom_id}")
                print(f"     Reference: {bom_reference}")
                print(f"     Posted V14 BOM ID ({bom_v14['id']}) in chatter")
                
            except Exception as e:
                print(f"  ❌ Failed to create BOM: {e}")
                failed_to_create.append({'bom_id': bom_v14['id'], 'error': str(e)})
        
        # Summary
        print("\n" + "=" * 80)
        print("SYNC SUMMARY")
        print("=" * 80)
        print(f"Total BOMs in V14: {len(boms_v14)}")
        print(f"BOMs deleted in V18: {deleted_count}")
        print(f"BOMs created in V18: {created_count}")
        print(f"Failed to create: {len(failed_to_create)}")
        print(f"Products created: {len(created_products)}")
        
        if created_products:
            print("\nProducts created in V18:")
            for prod in created_products:
                print(f"  - {prod['name']} (Code: {prod['code']}, ID: {prod['id']})")
        
        if failed_to_create:
            print("\nFailed to create BOMs:")
            for fail in failed_to_create:
                print(f"  - BOM ID (V14): {fail['bom_id']}, Error: {fail['error']}")
        
        print("=" * 80)
    
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()
